In [4]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from diffusion_policy.policy.diffusion_unet_image_policy import DiffusionUnetImagePolicy
from diffusion_policy.model.vision.multi_image_obs_encoder import MultiImageObsEncoder
from diffusion_policy.model.vision.timm_model import TimmRGBModel
from diffusers.schedulers.scheduling_ddim import DDIMScheduler
from diffusion_policy.dataset.real_image_dataset import RealImageDataset

In [20]:
n_latency_steps = 0
n_obs_steps = 2
n_action_steps = 8
horizon = 16

shape_meta = {
    "obs": {
        "camera_0": {
            "shape": [3, 224, 224],
            "type": "rgb"
        },
        "camera_1": {
            "shape": [3, 224, 224],
            "type": "rgb"
        },
        "robot_eef_pose_6d_rot": {
            "shape": [9],
            "type": "low_dim"
        },
        "gripper_width": {
            "shape": [1],
            "type": "low_dim"
        }
    },
    "action": {
        "shape": [10]
    }
}



from omegaconf import OmegaConf
shape_meta = OmegaConf.create(shape_meta)
dataset_path = "/home/tlips/Code/diffusion_policy/data/demo_place-cb"
from torchvision.transforms import Resize, RandomCrop, ColorJitter
dataset = RealImageDataset(shape_meta=shape_meta, dataset_path=dataset_path, image_buffer_resolution=[240,240],
 horizon=horizon, pad_before=n_obs_steps-1+n_latency_steps, pad_after=n_action_steps-1, n_obs_steps=n_obs_steps, n_latency_steps=n_latency_steps, 
use_cache=True, seed=42, val_ratio=0.1, max_train_episodes=None,
 delta_action=True,
 image_transforms=[Resize((240,240)),RandomCrop((224,224)), ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)])

Replay buffer shape meta:
{'obs': {'camera_0': {'shape': [3, 224, 224], 'type': 'rgb'}, 'camera_1': {'shape': [3, 224, 224], 'type': 'rgb'}, 'robot_eef_pose_6d_rot': {'shape': [9], 'type': 'low_dim'}, 'gripper_width': {'shape': [1], 'type': 'low_dim'}}, 'action': {'shape': [10]}}
Acquiring lock on cache.
Loading cached ReplayBuffer from Disk.
Loaded!
image_keys: ['camera_0', 'camera_1']


In [21]:
x = dataset[0]
print(x["obs"]["camera_0"].shape)

torch.Size([2, 3, 224, 224])


In [8]:
# profile dataset access
import cProfile

cProfile.run("dataset[0]", sort="cumtime")

         14979 function calls (14965 primitive calls) in 0.027 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.027    0.027 {built-in method builtins.exec}
        1    0.000    0.000    0.027    0.027 <string>:1(<module>)
        1    0.000    0.000    0.027    0.027 real_image_dataset.py:287(__getitem__)
        2    0.000    0.000    0.022    0.011 transforms.py:92(__call__)
        6    0.000    0.000    0.022    0.004 module.py:1124(_call_impl)
        2    0.000    0.000    0.020    0.010 transforms.py:1225(forward)
        2    0.000    0.000    0.018    0.009 functional.py:866(adjust_hue)
        2    0.000    0.000    0.018    0.009 functional_tensor.py:193(adjust_hue)
        2    0.001    0.000    0.015    0.007 functional_tensor.py:301(_hsv2rgb)
        2    0.000    0.000    0.012    0.006 functional.py:192(einsum)
        2    0.012    0.006    0.012    0.006 {built-in method

In [9]:
rgb = TimmRGBModel(model_name='resnet18',image_size=(224,224),pretrained=True)
obs_encoder = MultiImageObsEncoder(shape_meta=shape_meta,rgb_model=rgb)
noise_scheduler = DDIMScheduler(num_train_timesteps=100)
policy = DiffusionUnetImagePolicy(shape_meta=shape_meta,noise_scheduler=noise_scheduler,obs_encoder=obs_encoder,horizon=horizon,n_action_steps=n_action_steps,
                                  n_obs_steps=n_obs_steps,num_inference_steps=100)
policy.set_normalizer(dataset.get_normalizer())

warning, you should probably turn off EMA as it will cause performance issues with batchnorms. cannot replace batchnorms since you are using a pretrained model.


iterating dataset to get normalization:   3%|▎         | 14/498 [00:05<02:20,  3.45it/s]

dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6

iterating dataset to get normalization:  19%|█▊        | 93/498 [00:05<00:12, 32.42it/s]

dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6

...

iterating dataset to get normalization:  87%|████████▋ | 435/498 [00:07<00:00, 246.08it/s]

dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6

iterating dataset to get normalization:  94%|█████████▍| 467/498 [00:07<00:00, 255.45it/s]

dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6d_rot', 'gripper_width'])
dict_keys(['robot_eef_pose_6

iterating dataset to get normalization: 100%|██████████| 498/498 [00:07<00:00, 64.19it/s] 


In [10]:
# benchmark inference time using pytorch profiler

import torch.autograd.profiler as profiler
import torch
device="cuda:0"
policy.to(device)
policy.num_inference_steps = 16 # DDIM inference iterations
policy.eval()
import time 


obs =  dataset[0]["obs"]
obs = {k: torch.tensor(v).unsqueeze(0).to(device) for k,v in obs.items()}


/tmp/ipykernel_2048383/2042096983.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs = {k: torch.tensor(v).unsqueeze(0).to(device) for k,v in obs.items()}


In [11]:
def profile_inference():
    with torch.no_grad():
        with profiler.profile(record_shapes=True, profile_memory=True, use_cuda=True) as prof:
            with profiler.record_function("model_inference"):
                for i in range(1):
        
                    action = policy.predict_action(obs)
                    
        print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))
        
profile_inference()

INFO:2025-02-14 23:26:34 2048383:2048383 init.cpp:114] If you see CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, refer to https://developer.nvidia.com/nvidia-development-tools-solutions-err-nvgpuctrperm-cupti


-------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
              aten::cudnn_convolution        93.01%        1.785s        93.03%        1.786s       3.144ms        1.832s        95.47%        1.833s       3.228ms           0 b           0 b      44.41 Mb    -231.77 Mb           568  
                      model_inference         1.80%   

In [12]:
# profile training 
from torch.utils.data import DataLoader

device = "cuda:0"
policy.num_inference_steps = 100
policy.train()
policy.to(device)
dataloader = DataLoader(dataset, batch_size=64, num_workers=8, shuffle=False)

batch = next(iter(dataloader))
batch["obs"] = {k: torch.tensor(v).to(device) for k,v in batch["obs"].items()}

/tmp/ipykernel_2048383/1382257246.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["obs"] = {k: torch.tensor(v).to(device) for k,v in batch["obs"].items()}


In [13]:


# with profiler.profile(record_shapes=False, profile_memory=False, use_cuda=False) as prof:
#     with profiler.record_function("model_training"):
#         for i in range(2):
#             # mark in the profiler the forward pass
#             batch = next(iter(dataloader))
#             loss = policy.compute_loss(batch)
#             loss.backward()

# print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
# # dump trace


In [22]:
# create dataloaders with different numbers of workers, and compare the performance for loading the entire dataset 
import time
batch_size = 64
# create smaller dataset split
split,_= torch.utils.data.random_split(dataset, [10000, len(dataset)-10000])
print(len(split)//batch_size)
import tqdm

def do_one_pass(dataloader):
    avg_time = 0
    start = time.time()
    load_times = []
    transfer_times = []
    forward_pass_times = []
    backward_pass_times = []

    stop_time = time.time()
    for i, batch in enumerate(tqdm.tqdm(dataloader)):
        loaded_time = time.time()
        for k in batch["obs"]:
            batch["obs"][k] = batch["obs"][k].to(device)
        batch["action"] = batch["action"].to(device)
        transferred_time = time.time()
        loss = policy.compute_loss(batch)
        forwarded_time = time.time()
        loss.backward()
        backwared_time = time.time()

        load_times.append(loaded_time-stop_time)
        transfer_times.append(transferred_time-loaded_time)
        forward_pass_times.append(forwarded_time-transferred_time)
        backward_pass_times.append(backwared_time-forwarded_time)
        stop_time = time.time()

        # add average times to the tqdm description
        if i%10 == 0:
            tqdm.tqdm.write(f"avg load time: {sum(load_times)/len(load_times)}, avg transfer time: {sum(transfer_times)/len(transfer_times)}, avg forward pass time: {sum(forward_pass_times)/len(forward_pass_times)}, avg backward pass time: {sum(backward_pass_times)/len(backward_pass_times)}")

    print(f"avg load time: {sum(load_times)/len(load_times)}, avg transfer time: {sum(transfer_times)/len(transfer_times)}, avg forward pass time: {sum(forward_pass_times)/len(forward_pass_times)}, avg backward pass time: {sum(backward_pass_times)/len(backward_pass_times)}")

    return time.time()-start

for n_workers in [8]:
    dataloader = DataLoader(split, batch_size=batch_size, num_workers=n_workers, shuffle=True, pin_memory=False,persistent_workers=True)
    print(f"n_workers: {n_workers}, time: {do_one_pass(dataloader)}")


156


  0%|          | 0/157 [00:00<?, ?it/s]

  1%|          | 1/157 [00:05<14:12,  5.47s/it]

avg load time: 5.302886486053467, avg transfer time: 0.07214236259460449, avg forward pass time: 0.05287671089172363, avg backward pass time: 0.03538036346435547


  7%|▋         | 11/157 [00:09<01:16,  1.90it/s]

...

avg load time: 0.25156979696125004, avg transfer time: 0.12125991253142661, avg forward pass time: 0.049379998065055684, avg backward pass time: 0.02991788776208323


 97%|█████████▋| 152/157 [01:06<00:01,  4.91it/s]

avg load time: 0.24210178931027848, avg transfer time: 0.12046184129272865, avg forward pass time: 0.049125188233836596, avg backward pass time: 0.02938002466365991


100%|██████████| 157/157 [01:08<00:00,  2.30it/s]

avg load time: 0.23542438494931361, avg transfer time: 0.11922295382068415, avg forward pass time: 0.04869802438529434, avg backward pass time: 0.028978886877655225
n_workers: 8, time: 68.28810906410217


In [15]:
import time 
start = time.time()
d = {k: torch.rand(2,3,224,224).to(device) for k in  range(4)}
print(time.time()-start)

0.004042625427246094


In [16]:
# cprofile a single dataloader iteration
import cProfile
dataloader = DataLoader(dataset, batch_size=64, num_workers=0, shuffle=False)
with cProfile.Profile() as pr:
    c = 0
    for batch in dataloader:
        c += 1
        if c > 10:
            break
       
    

pr.print_stats(sort="cumtime")

         10717423 function calls (10707488 primitive calls) in 17.746 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       11    0.000    0.000   17.751    1.614 dataloader.py:676(__next__)
       11    0.002    0.000   17.750    1.614 dataloader.py:719(_next_data)
       11    0.000    0.000   17.748    1.613 fetch.py:47(fetch)
       11    0.005    0.000   17.419    1.584 fetch.py:49(<listcomp>)
      704    0.115    0.000   17.414    0.025 real_image_dataset.py:286(__getitem__)
     1408    0.009    0.000   14.020    0.010 transforms.py:92(__call__)
     4224    0.013    0.000   14.011    0.003 module.py:1124(_call_impl)
     1408    0.059    0.000   12.465    0.009 transforms.py:1225(forward)
     1408    0.007    0.000   10.999    0.008 functional.py:866(adjust_hue)
     1408    0.172    0.000   10.989    0.008 functional_tensor.py:193(adjust_hue)
     1408    0.672    0.000    8.703    0.006 functional_tensor.py:3